### 5.3 变量管理
创建变量

方法：tf.Variable 和 tf.get_variable

区别：tf.Variable中变量名name是一个可选参数。tf.get_variable中name是必须的，创建失败会报错，避免无意识的变量复用造成的错误。

获取变量：tf.get_variable 和 tf.variable_scope

In [1]:
import tensorflow as tf
# 创建变量的两种等价方式
# get_variable即可获取变量，又可创建变量
vv = tf.get_variable("vv", shape=[1],
                   initializer=tf.constant_initializer(1.0))
vv = tf.Variable(tf.constant(1.0, shape=[1]), name="vv")

In [1]:
import tensorflow as tf

# var1是一个数值
var1 = tf.get_variable("var1", shape=None, initializer=tf.constant(1.0))
# var2是一个一维数组
var2 = tf.Variable(tf.constant(1.0, shape=[1], dtype=tf.float32), name="var2")

#即使设置相同的name，实际上也是不同的变量
var3 = tf.Variable(tf.constant(1.0, shape=[1], dtype=tf.float32), name="var")
var4 = tf.Variable(1.0, name="var")
print("var1.name:", var1.name)
print("var2.name:", var2.name)
print("var3.name:", var3.name)
print("var4.name:", var4.name)

#变量初始化函数
var5 = tf.Variable(tf.random_normal(shape=[2, 3], mean=0.5, stddev=1.0, seed=1.0), name='var5')
var6 = tf.Variable(tf.ones(shape=[2, 2]), name='var6')
var7 = tf.Variable(tf.zeros(shape=[2, 1]), name='var7')

with tf.Session() as sess:
    #变量赋值函数，会自动创建一个operation，必须执行operation，否则不会进行赋值
    # var1是一个数值
    assign_op1 = tf.assign(var1, 2.5)
    # var2是一个一维数组
    assign_op2 = var2.assign([3.5])

    sess.run(tf.global_variables_initializer())
    print("before update var1:", var1.eval())
    sess.run(assign_op1)
    print("after update var1:", var1.eval())
    print("before update var2:", var2.eval())
    sess.run(assign_op2)
    print("after update var2:", var2.eval())
    print("var5:", var5.eval())
    print("var6:", var6.eval())
    print("var7:", var7.eval())


var1.name: var1:0
var2.name: var2:0
var3.name: var:0
var4.name: var_1:0
before update var1: 1.0
after update var1: 2.5
before update var2: [1.]
after update var2: [3.5]
var5: [[-0.31131822  1.9845988   0.5653294 ]
 [-1.9427042   0.5992484   1.0912243 ]]
var6: [[1. 1.]
 [1. 1.]]
var7: [[0.]
 [0.]]


#### 1. 在上下文管理器“foo”中创建变量“v”。

In [2]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", shape=[1], initializer=tf.constant_initializer(1.0))

# 因为在命名空间foo中以及存在名字为v的变量，下面代码会报错
#with tf.variable_scope("foo"):
   # v = tf.get_variable("v", [1])

# 将reuse设为true，这样可直接获取已声明的变量
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", shape=[1])
print(v == v1)
print("v.name", v.name)
print("v1.name", v1.name)

# 如果reuse设置为True, tf.variable_scope将只能获取已经创建过的变量
# 因为在命名空间bar中还没有创建变量v2，所以会报错
with tf.variable_scope("bar", reuse=False):
    v = tf.get_variable("v2", shape=[1])

print("v.name", v.name)


True
v.name foo/v:0
v1.name foo/v:0
v.name bar/v2:0


#### 2. 嵌套上下文管理器中的reuse参数的使用。

In [3]:
with tf.variable_scope("root"):
    print(tf.get_variable_scope().reuse)
    
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        
        with tf.variable_scope("bar"):
            print(tf.get_variable_scope().reuse)
            
    print(tf.get_variable_scope().reuse)


False
True
True
False


#### 3. 通过variable_scope来管理变量。

In [4]:
v1 = tf.get_variable("v", [1])
print("v1.name:", v1.name)

with tf.variable_scope("foo",reuse=True):
    v2 = tf.get_variable("v", [1])
print("v2.name:", v2.name)

# 变量范围嵌套
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v", [1])
        print("v3.name:", v3.name)
        
v4 = tf.get_variable("v1", [1])
print("v4.name:", v4.name)

v1.name: v:0
v2.name: foo/v:0
v3.name: foo/bar/v:0
v4.name: v1:0


#### 4. 我们可以通过变量的名称来获取变量。

In [5]:
with tf.variable_scope("",reuse=True):
    v5 = tf.get_variable("foo/bar/v", [1])
    print("v5 == v3:", v5 == v3)
    v6 = tf.get_variable("v1", [1])     
    print("v6 == v4:", v6 == v4)

v5 == v3: True
v6 == v4: True
